# Sequential DQN

In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

/home/aras/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

>**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import gym

# Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [3]:
import numpy as np
state = env.reset()
for _ in range(10):
    # env.render()
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action) # take a random action
    #print('state, action, next_state, reward, done, info:', state, action, next_state, reward, done, info)
    state = next_state
    if done:
        state = env.reset()

To shut the window showing the simulation, use `env.close()`.

If you ran the simulation above, we can look at the rewards:

In [4]:
# print(rewards[-20:])
# print(np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
# print(np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
# print(np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
# print(np.max(np.array(rewards)), np.min(np.array(rewards)))
# print(np.max(np.array(states)), np.min(np.array(states)))

The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [5]:
def model_input(state_size, hidden_size, batch_size=1):
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    # RNN
    #cell = tf.nn.rnn_cell.GRUCell(hidden_size)
    #cell = tf.nn.rnn_cell.LSTMCell(hidden_size) #??? NOT working
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell], state_is_tuple=True)
    initial_state = cells.zero_state(batch_size, tf.float32)
    return states, actions, targetQs, cells, initial_state

In [6]:
# RNN generator or sequence generator
def generator(states, action_size, initial_state, cells, hidden_size, reuse=False): 
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        inputs = tf.layers.dense(inputs=states, units=hidden_size)
        print(states.shape, inputs.shape)
        
        # with tf.variable_scope('dynamic_rnn_', reuse=tf.AUTO_REUSE):
        # dynamic means adapt to the batch_size and
        # static means can NOT adapt to the batch_size
        inputs_rnn = tf.reshape(inputs, [1, -1, hidden_size]) # NxH -> 1xNxH
        print(inputs_rnn.shape, initial_state)
        outputs_rnn, final_state = tf.nn.dynamic_rnn(cell=cells, inputs=inputs_rnn, initial_state=initial_state)
        print(outputs_rnn.shape, final_state)
        outputs = tf.reshape(outputs_rnn, [-1, hidden_size]) # 1xNxH -> NxH
        print(outputs.shape)

        # Last fully connected layer
        logits = tf.layers.dense(inputs=outputs, units=action_size)
        print(logits.shape)
        #predictions = tf.nn.softmax(logits)
        
        # logits are the action logits
        return logits, final_state

In [7]:
def model_loss(action_size, hidden_size, states, cells, initial_state, actions, targetQs):
    actions_logits, final_state = generator(states=states, cells=cells, initial_state=initial_state, 
                                            hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    Qs = tf.reduce_max(actions_logits*actions_labels, axis=1)
    loss = tf.reduce_mean(tf.square(Qs - targetQs))
    return actions_logits, final_state, loss

In [8]:
def model_opt(loss, learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]

    # # Optimize MLP/CNN
    # with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
    # #opt = tf.train.AdamOptimizer(learning_rate).minimize(loss, var_list=g_vars)

    # # Optimize RNN
    #grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(loss, g_vars), clip_norm=5) # usually around 1-5
    grads = tf.gradients(loss, g_vars)
    opt = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads_and_vars=zip(grads, g_vars))

    return opt

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs, cells, self.initial_state = model_input(
                state_size=state_size, hidden_size=hidden_size)
        
        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.final_state, self.loss = model_loss(
            action_size=action_size, hidden_size=hidden_size, 
            states=self.states, actions=self.actions, 
            targetQs=self.targetQs, cells=cells, initial_state=self.initial_state)

        # Update the model: backward pass and backprop
        self.opt = model_opt(loss=self.loss, learning_rate=learning_rate)

## Experience replay

Reinforcement learning algorithms can have stability issues due to correlations between states. To reduce correlations when training, we can store the agent's experiences and later draw a random mini-batch of those experiences to train on. 

Here, we'll create a `Memory` object that will store our experiences, our transitions $<s, a, r, s'>$. This memory will have a maxmium capacity, so we can keep newer experiences in memory while getting rid of older experiences. Then, we'll sample a random mini-batch of transitions $<s, a, r, s'>$ and train on those.

Below, I've implemented a `Memory` object. If you're unfamiliar with `deque`, this is a double-ended queue. You can think of it like a tube open on both sides. You can put objects in either side of the tube. But if it's full, adding anything more will push an object out the other side. This is a great data structure to use for the memory buffer.

In [10]:
from collections import deque

class Memory():    
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
        self.states = deque(maxlen=max_size)

## Exploration - Exploitation

To learn about the environment and rules of the game, the agent needs to explore by taking random actions. We'll do this by choosing a random action with some probability $\epsilon$ (epsilon).  That is, with some probability $\epsilon$ the agent will make a random action and with probability $1 - \epsilon$, the agent will choose an action from $Q(s,a)$. This is called an **$\epsilon$-greedy policy**.


At first, the agent needs to do a lot of exploring. Later when it has learned more, the agent can favor choosing actions based on what it has learned. This is called _exploitation_. We'll set it up so the agent is more likely to explore early in training, then more likely to exploit later in training.

## Q-Learning training algorithm

Putting all this together, we can list out the algorithm we'll use to train the network. We'll train the network in _episodes_. One *episode* is one simulation of the game. For this game, the goal is to keep the pole upright for 195 frames. So we can start a new episode once meeting that goal. The game ends if the pole tilts over too far, or if the cart moves too far the left or right. When a game ends, we'll start a new episode. Now, to train the agent:

* Initialize the memory $D$
* Initialize the action-value network $Q$ with random weights
* **For** episode = 1, $M$ **do**
  * **For** $t$, $T$ **do**
     * With probability $\epsilon$ select a random action $a_t$, otherwise select $a_t = \mathrm{argmax}_a Q(s,a)$
     * Execute action $a_t$ in simulator and observe reward $r_{t+1}$ and new state $s_{t+1}$
     * Store transition $<s_t, a_t, r_{t+1}, s_{t+1}>$ in memory $D$
     * Sample random mini-batch from $D$: $<s_j, a_j, r_j, s'_j>$
     * Set $\hat{Q}_j = r_j$ if the episode ends at $j+1$, otherwise set $\hat{Q}_j = r_j + \gamma \max_{a'}{Q(s'_j, a')}$
     * Make a gradient descent step with loss $(\hat{Q}_j - Q(s_j, a_j))^2$
  * **endfor**
* **endfor**

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [11]:
# Network parameters
action_size = 2
state_size = 4
hidden_size = 64               # number of units in each Q-network hidden layer
learning_rate = 0.0001         # Q-network learning rate

# Memory parameters
memory_size = 128            # memory capacity - 1000 DQN
batch_size = 128             # experience mini-batch size - 20 DQN
gamma = 0.99                 # future reward discount

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

# Init the memory
memory = Memory(max_size=batch_size)

(?, 4) (?, 64)
(1, ?, 64) (LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros:0' shape=(1, 64) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(1, 64) dtype=float32>),)
(1, ?, 64) (LSTMStateTuple(c=<tf.Tensor 'generator/rnn/while/Exit_3:0' shape=(1, 64) dtype=float32>, h=<tf.Tensor 'generator/rnn/while/Exit_4:0' shape=(1, 64) dtype=float32>),)
(?, 64)
(?, 2)


In [13]:
model.initial_state[0]

LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros:0' shape=(1, 64) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(1, 64) dtype=float32>)

## Populate the memory (exprience memory)

Here I'm re-initializing the simulation and pre-populating the memory. The agent is taking random actions and storing the transitions in memory. This will help the agent with exploring the game.

In [14]:
state = env.reset()
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done)])
    #memory.states.append(np.zeros([1, hidden_size])) # gru
    memory.states.append([np.zeros([1, hidden_size]), np.zeros([1, hidden_size])]) # lstm
    state = next_state
    if done is True:
        # Reseting the env/first state
        state = env.reset()

In [15]:
# # Training
# batch = memory.buffer
# states = np.array([each[0] for each in batch])
# actions = np.array([each[1] for each in batch])
# next_states = np.array([each[2] for each in batch])
# rewards = np.array([each[3] for each in batch])
# dones = np.array([each[4] for each in batch])

In [16]:
# memory.states[0].shape, model.initial_state[0].shape # gru
memory.states[0][1].shape, model.initial_state[0][1].shape #lstm

((1, 64), TensorShape([Dimension(1), Dimension(64)]))

In [17]:
# memory.states[0][0].shape, model.initial_state[0][0].shape

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [18]:
# initial_states = np.array(memory.states)
# initial_states.shape

In [19]:
saver = tf.train.Saver()
episode_rewards_list, rewards_list, loss_list = [], [], []

# TF session for training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
    
    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0
        loss_batch = []
        state = env.reset()
        initial_state = sess.run(model.initial_state)

        # Training steps/batches
        while True:
            action_logits, final_state = sess.run([model.actions_logits, model.final_state],
                                                  feed_dict = {model.states: state.reshape([1, -1]), 
                                                               model.initial_state: initial_state})
            action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done)])
            memory.states.append(initial_state)
            total_reward += reward
            state = next_state
            initial_state = final_state

            # Training
            batch = memory.buffer
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            next_states = np.array([each[2] for each in batch])
            rewards = np.array([each[3] for each in batch])
            dones = np.array([each[4] for each in batch])
            initial_states = memory.states
            next_actions_logits = sess.run(model.actions_logits, 
                                           feed_dict = {model.states: next_states,
                                                        model.initial_state: initial_states[1]})
            nextQs = np.max(next_actions_logits, axis=1) * (1-dones)
            targetQs = rewards + (gamma * nextQs)
            loss, _ = sess.run([model.loss, model.opt], feed_dict = {model.states: states, 
                                                                     model.actions: actions,
                                                                     model.targetQs: targetQs,
                                                                     model.initial_state: initial_states[0]})
            # End of training
            loss_batch.append(loss)
            if done is True:
                break
                
        # Outputing: priting out/Potting
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'loss:{:.4f}'.format(np.mean(loss_batch)))
        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        loss_list.append([ep, np.mean(loss_batch)])
        # Break episode/epoch loop
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:9.0000 R:9.0000 loss:1.0361
Episode:1 meanR:9.0000 R:9.0000 loss:1.0564
Episode:2 meanR:9.0000 R:9.0000 loss:1.0656
Episode:3 meanR:8.7500 R:8.0000 loss:1.0961
Episode:4 meanR:8.8000 R:9.0000 loss:1.1396
Episode:5 meanR:8.6667 R:8.0000 loss:1.1949
Episode:6 meanR:8.8571 R:10.0000 loss:1.2547
Episode:7 meanR:9.0000 R:10.0000 loss:1.3066
Episode:8 meanR:8.8889 R:8.0000 loss:1.3911
Episode:9 meanR:8.9000 R:9.0000 loss:1.3862
Episode:10 meanR:9.0000 R:10.0000 loss:1.4211
Episode:11 meanR:9.0000 R:9.0000 loss:1.5247
Episode:12 meanR:9.0000 R:9.0000 loss:1.6145
Episode:13 meanR:9.0000 R:9.0000 loss:1.8368
Episode:14 meanR:9.0667 R:10.0000 loss:2.0206
Episode:15 meanR:9.0000 R:8.0000 loss:2.2681
Episode:16 meanR:9.0000 R:9.0000 loss:2.5478
Episode:17 meanR:9.0000 R:9.0000 loss:2.8759
Episode:18 meanR:8.9474 R:8.0000 loss:3.1003
Episode:19 meanR:8.9500 R:9.0000 loss:3.2063
Episode:20 meanR:8.9524 R:9.0000 loss:3.5263
Episode:21 meanR:8.9545 R:9.0000 loss:3.7069
Episode:22 meanR

Episode:179 meanR:9.3900 R:10.0000 loss:0.1023
Episode:180 meanR:9.3900 R:10.0000 loss:0.0949
Episode:181 meanR:9.3900 R:10.0000 loss:0.0919
Episode:182 meanR:9.3900 R:9.0000 loss:0.0895
Episode:183 meanR:9.4000 R:9.0000 loss:0.0929
Episode:184 meanR:9.4000 R:9.0000 loss:0.0898
Episode:185 meanR:9.3900 R:9.0000 loss:0.0978
Episode:186 meanR:9.4000 R:9.0000 loss:0.1017
Episode:187 meanR:9.3900 R:9.0000 loss:0.1019
Episode:188 meanR:9.4000 R:10.0000 loss:0.0856
Episode:189 meanR:9.4000 R:9.0000 loss:0.0828
Episode:190 meanR:9.4100 R:10.0000 loss:0.0741
Episode:191 meanR:9.4200 R:10.0000 loss:0.0660
Episode:192 meanR:9.4200 R:9.0000 loss:0.0650
Episode:193 meanR:9.4200 R:10.0000 loss:0.0603
Episode:194 meanR:9.4200 R:10.0000 loss:0.0677
Episode:195 meanR:9.4200 R:10.0000 loss:0.0630
Episode:196 meanR:9.4200 R:10.0000 loss:0.0622
Episode:197 meanR:9.4200 R:10.0000 loss:0.0573
Episode:198 meanR:9.4100 R:8.0000 loss:0.0659
Episode:199 meanR:9.4100 R:9.0000 loss:0.0724
Episode:200 meanR:9.420

Episode:356 meanR:9.2800 R:9.0000 loss:0.0672
Episode:357 meanR:9.2600 R:8.0000 loss:0.0697
Episode:358 meanR:9.2600 R:10.0000 loss:0.0629
Episode:359 meanR:9.2600 R:9.0000 loss:0.0599
Episode:360 meanR:9.2500 R:9.0000 loss:0.0588
Episode:361 meanR:9.2500 R:9.0000 loss:0.0548
Episode:362 meanR:9.2600 R:10.0000 loss:0.0540
Episode:363 meanR:9.2600 R:9.0000 loss:0.0565
Episode:364 meanR:9.2700 R:10.0000 loss:0.0566
Episode:365 meanR:9.2800 R:10.0000 loss:0.0670
Episode:366 meanR:9.2600 R:8.0000 loss:0.0574
Episode:367 meanR:9.2400 R:8.0000 loss:0.0418
Episode:368 meanR:9.2300 R:10.0000 loss:0.0563
Episode:369 meanR:9.2200 R:9.0000 loss:0.0734
Episode:370 meanR:9.2100 R:9.0000 loss:0.0508
Episode:371 meanR:9.2000 R:9.0000 loss:0.0576
Episode:372 meanR:9.2000 R:10.0000 loss:0.0446
Episode:373 meanR:9.1800 R:8.0000 loss:0.0436
Episode:374 meanR:9.1800 R:10.0000 loss:0.0571
Episode:375 meanR:9.2000 R:10.0000 loss:0.0518
Episode:376 meanR:9.2000 R:9.0000 loss:0.0533
Episode:377 meanR:9.2100 R

Episode:533 meanR:9.3300 R:9.0000 loss:0.0542
Episode:534 meanR:9.3300 R:10.0000 loss:0.0533
Episode:535 meanR:9.3500 R:11.0000 loss:0.0444
Episode:536 meanR:9.3600 R:10.0000 loss:0.0468
Episode:537 meanR:9.3700 R:10.0000 loss:0.0415
Episode:538 meanR:9.3500 R:8.0000 loss:0.0440
Episode:539 meanR:9.3600 R:10.0000 loss:0.0435
Episode:540 meanR:9.3800 R:10.0000 loss:0.0453
Episode:541 meanR:9.4000 R:10.0000 loss:0.0420
Episode:542 meanR:9.4000 R:10.0000 loss:0.0479
Episode:543 meanR:9.4000 R:8.0000 loss:0.0566
Episode:544 meanR:9.3800 R:8.0000 loss:0.0613
Episode:545 meanR:9.3900 R:11.0000 loss:0.0505
Episode:546 meanR:9.3900 R:9.0000 loss:0.0552
Episode:547 meanR:9.3900 R:10.0000 loss:0.0530
Episode:548 meanR:9.3900 R:8.0000 loss:0.0571
Episode:549 meanR:9.3800 R:9.0000 loss:0.0532
Episode:550 meanR:9.3900 R:9.0000 loss:0.0527
Episode:551 meanR:9.3900 R:10.0000 loss:0.0540
Episode:552 meanR:9.3800 R:9.0000 loss:0.0573
Episode:553 meanR:9.3800 R:9.0000 loss:0.0464
Episode:554 meanR:9.390

Episode:710 meanR:9.4200 R:10.0000 loss:0.0391
Episode:711 meanR:9.4400 R:11.0000 loss:0.0332
Episode:712 meanR:9.4400 R:9.0000 loss:0.0547
Episode:713 meanR:9.4300 R:9.0000 loss:0.0435
Episode:714 meanR:9.4300 R:9.0000 loss:0.0426
Episode:715 meanR:9.4500 R:10.0000 loss:0.0522
Episode:716 meanR:9.4600 R:9.0000 loss:0.0453
Episode:717 meanR:9.4600 R:10.0000 loss:0.0385
Episode:718 meanR:9.4500 R:9.0000 loss:0.0467
Episode:719 meanR:9.4500 R:10.0000 loss:0.0416
Episode:720 meanR:9.4400 R:8.0000 loss:0.0434
Episode:721 meanR:9.4400 R:9.0000 loss:0.0372
Episode:722 meanR:9.4300 R:9.0000 loss:0.0416
Episode:723 meanR:9.4200 R:9.0000 loss:0.0323
Episode:724 meanR:9.4200 R:10.0000 loss:0.0309
Episode:725 meanR:9.4400 R:11.0000 loss:0.0267
Episode:726 meanR:9.4400 R:10.0000 loss:0.0281
Episode:727 meanR:9.4400 R:9.0000 loss:0.0279
Episode:728 meanR:9.4400 R:10.0000 loss:0.0295
Episode:729 meanR:9.4500 R:11.0000 loss:0.0251
Episode:730 meanR:9.4400 R:9.0000 loss:0.0329
Episode:731 meanR:9.4400

...

Episode:10396 meanR:9.4800 R:10.0000 loss:0.0060
Episode:10397 meanR:9.4800 R:9.0000 loss:0.0080
Episode:10398 meanR:9.4700 R:9.0000 loss:0.0087
Episode:10399 meanR:9.4600 R:9.0000 loss:0.0058
Episode:10400 meanR:9.4500 R:9.0000 loss:0.0061
Episode:10401 meanR:9.4500 R:9.0000 loss:0.0063
Episode:10402 meanR:9.4500 R:10.0000 loss:0.0048
Episode:10403 meanR:9.4500 R:9.0000 loss:0.0037
Episode:10404 meanR:9.4400 R:8.0000 loss:0.0036
Episode:10405 meanR:9.4500 R:10.0000 loss:0.0093
Episode:10406 meanR:9.4500 R:9.0000 loss:0.0098
Episode:10407 meanR:9.4500 R:10.0000 loss:0.0089
Episode:10408 meanR:9.4500 R:10.0000 loss:0.0084
Episode:10409 meanR:9.4400 R:9.0000 loss:0.0087
Episode:10410 meanR:9.4400 R:10.0000 loss:0.0078
Episode:10411 meanR:9.4500 R:9.0000 loss:0.0078
Episode:10412 meanR:9.4600 R:10.0000 loss:0.0084
Episode:10413 meanR:9.4700 R:10.0000 loss:0.0076
Episode:10414 meanR:9.4600 R:8.0000 loss:0.0079
Episode:10415 meanR:9.4500 R:8.0000 loss:0.0089
Episode:10416 meanR:9.4700 R:10.

Episode:10565 meanR:9.4400 R:10.0000 loss:0.0021
Episode:10566 meanR:9.4400 R:9.0000 loss:0.0029
Episode:10567 meanR:9.4200 R:9.0000 loss:0.0043
Episode:10568 meanR:9.4200 R:9.0000 loss:0.0049
Episode:10569 meanR:9.4000 R:8.0000 loss:0.0040
Episode:10570 meanR:9.4100 R:10.0000 loss:0.0043
Episode:10571 meanR:9.4000 R:9.0000 loss:0.0044
Episode:10572 meanR:9.3900 R:8.0000 loss:0.0053
Episode:10573 meanR:9.3900 R:10.0000 loss:0.0043
Episode:10574 meanR:9.3900 R:9.0000 loss:0.0044
Episode:10575 meanR:9.3800 R:8.0000 loss:0.0053
Episode:10576 meanR:9.3700 R:8.0000 loss:0.0057
Episode:10577 meanR:9.3700 R:9.0000 loss:0.0058
Episode:10578 meanR:9.3700 R:9.0000 loss:0.0061
Episode:10579 meanR:9.3800 R:10.0000 loss:0.0059
Episode:10580 meanR:9.3700 R:9.0000 loss:0.0053
Episode:10581 meanR:9.3800 R:10.0000 loss:0.0040
Episode:10582 meanR:9.3800 R:9.0000 loss:0.0036
Episode:10583 meanR:9.3800 R:9.0000 loss:0.0047
Episode:10584 meanR:9.3700 R:9.0000 loss:0.0090
Episode:10585 meanR:9.3600 R:9.0000

Episode:10735 meanR:9.4000 R:8.0000 loss:0.0044
Episode:10736 meanR:9.3800 R:8.0000 loss:0.0050
Episode:10737 meanR:9.3600 R:8.0000 loss:0.0059
Episode:10738 meanR:9.3600 R:10.0000 loss:0.0078
Episode:10739 meanR:9.3500 R:8.0000 loss:0.0074
Episode:10740 meanR:9.3300 R:8.0000 loss:0.0061
Episode:10741 meanR:9.3400 R:10.0000 loss:0.0053
Episode:10742 meanR:9.3500 R:9.0000 loss:0.0055
Episode:10743 meanR:9.3500 R:10.0000 loss:0.0061
Episode:10744 meanR:9.3300 R:8.0000 loss:0.0054
Episode:10745 meanR:9.3200 R:10.0000 loss:0.0047
Episode:10746 meanR:9.3200 R:9.0000 loss:0.0048
Episode:10747 meanR:9.3100 R:9.0000 loss:0.0064
Episode:10748 meanR:9.3100 R:9.0000 loss:0.0062
Episode:10749 meanR:9.3000 R:9.0000 loss:0.0060
Episode:10750 meanR:9.3000 R:9.0000 loss:0.0063
Episode:10751 meanR:9.3000 R:10.0000 loss:0.0108
Episode:10752 meanR:9.2900 R:8.0000 loss:0.0043
Episode:10753 meanR:9.2900 R:8.0000 loss:0.0040
Episode:10754 meanR:9.3000 R:11.0000 loss:0.0048
Episode:10755 meanR:9.3000 R:9.000

Episode:10905 meanR:9.3800 R:10.0000 loss:0.0064
Episode:10906 meanR:9.3700 R:10.0000 loss:0.0104
Episode:10907 meanR:9.3600 R:10.0000 loss:0.0067
Episode:10908 meanR:9.3600 R:10.0000 loss:0.0053
Episode:10909 meanR:9.3700 R:10.0000 loss:0.0065
Episode:10910 meanR:9.3800 R:10.0000 loss:0.0068
Episode:10911 meanR:9.3900 R:10.0000 loss:0.0085
Episode:10912 meanR:9.3800 R:8.0000 loss:0.0088
Episode:10913 meanR:9.3900 R:10.0000 loss:0.0079
Episode:10914 meanR:9.3900 R:9.0000 loss:0.0092
Episode:10915 meanR:9.3900 R:9.0000 loss:0.0084
Episode:10916 meanR:9.3900 R:10.0000 loss:0.0081
Episode:10917 meanR:9.3800 R:9.0000 loss:0.0066
Episode:10918 meanR:9.4000 R:11.0000 loss:0.0058
Episode:10919 meanR:9.4200 R:10.0000 loss:0.0057
Episode:10920 meanR:9.4000 R:8.0000 loss:0.0055
Episode:10921 meanR:9.4000 R:10.0000 loss:0.0068
Episode:10922 meanR:9.3900 R:8.0000 loss:0.0066
Episode:10923 meanR:9.4000 R:10.0000 loss:0.0060
Episode:10924 meanR:9.4100 R:9.0000 loss:0.0063
Episode:10925 meanR:9.3900 

Episode:11075 meanR:9.2800 R:10.0000 loss:0.0029
Episode:11076 meanR:9.2700 R:8.0000 loss:0.0055
Episode:11077 meanR:9.2600 R:8.0000 loss:0.0127
Episode:11078 meanR:9.2500 R:9.0000 loss:0.0069
Episode:11079 meanR:9.2400 R:9.0000 loss:0.0138
Episode:11080 meanR:9.2600 R:10.0000 loss:0.0152
Episode:11081 meanR:9.2600 R:9.0000 loss:0.0109
Episode:11082 meanR:9.2800 R:11.0000 loss:0.0246
Episode:11083 meanR:9.2700 R:8.0000 loss:0.0150
Episode:11084 meanR:9.2700 R:8.0000 loss:0.0196
Episode:11085 meanR:9.2600 R:10.0000 loss:0.0283
Episode:11086 meanR:9.2600 R:10.0000 loss:0.0302
Episode:11087 meanR:9.2700 R:10.0000 loss:0.0241
Episode:11088 meanR:9.2700 R:9.0000 loss:0.0258
Episode:11089 meanR:9.2800 R:9.0000 loss:0.0366
Episode:11090 meanR:9.2900 R:10.0000 loss:0.0226
Episode:11091 meanR:9.2800 R:9.0000 loss:0.0334
Episode:11092 meanR:9.2700 R:9.0000 loss:0.0474
Episode:11093 meanR:9.2900 R:10.0000 loss:0.0309
Episode:11094 meanR:9.3000 R:10.0000 loss:0.0203
Episode:11095 meanR:9.2800 R:9.

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Average losses')

## Testing

Let's checkout how our trained agent plays the game.

In [38]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episode/epoch
    for _ in range(10):
        total_reward = 0
        state = env.reset()
        initial_state = sess.run(model.initial_state) # Qs or current batch or states[:-1]
        
        # Steps/batches
        while True:
            env.render()
            action_logits, initial_state = sess.run([model.actions_logits, model.final_state],
                                                    feed_dict = {model.states: state.reshape([1, -1]), 
                                                                 model.initial_state: initial_state})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                break
        # At the end of each episode
        print('total_reward:{}'.format(total_reward))

# Close the env
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.